# Scrittopoli - Giornata 1

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk
import matplotlib.mlab as mlab

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import string

import mylib.spreadsheet as ms
import mylib.elo as melo
import mylib.utility as mu


In [2]:
day="20171214_142354" # definitivo
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day

calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])




## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [3]:
quadro=calendario.loc[1].copy()

riposano=quadro[quadro["squadra 2"]=="(riposo)"][["squadra 1"]]
riposano



,,squadra 1
girone,partita,
A,2,Gli ultimi


In [4]:
quadro=quadro[quadro["squadra 2"]!="(riposo)"]
quadro["vs"]="contro"

quadro=quadro[["squadra 1","vs","squadra 2"]]

quadro=quadro.sort_index()
quadro

squadra 1      vs           squadra 2
girone partita                                                           
A      1                          D Euforiche  contro        La threesome
B      1                          Green House  contro          I Bananari
       2        Gli scrittori del quartierino  contro      the 3 BrOthers
C      1                        Living Stones  contro  Retroguardia Trash
       2                          Le TreMende  contro  Armata Brancaleone

In [5]:
importlib.reload(mu)
mu.to_table_html_multi(quadro)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>La threesome</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>I Bananari</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>the 3 BrOthers</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Retroguardia Trash</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Le TreMende</td>
<td style='text

## Fase 1. Dalle formazioni ai match

In [6]:
squadre=scrittopoli.copy()
squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

importlib.reload(ms)
s=ms.N2017Spreadsheet()

old_data=s.get_formazioni(1)
s.set_formazioni(1,riposano["squadra 1"].values,squadre.index.unique(),old_data)


#formazioni_t=s.get_formazioni(1)
#formazioni_t



In [7]:
# Versione random di prova, da sostituire con lettura della formazione via spreadsheet sotto
#data=[]
#for sq in squadre.index.unique():
#    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
#    data.append([sq,capitano,riserva]+titolari)
#formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])
#formazioni=formazioni.set_index("squadra")
#formazioni["riposa"]=""
#formazioni.set_value("Gli ultimi","riposa","(riposa)")

formazioni=s.get_formazioni(1)


In [8]:
formazioni

,riposa,capitano,riserva,titolare 1,titolare 2
squadra,,,,,
Green House,,Bango Skank,Plata,Bango Skank,Joyopi
La threesome,,mina99,mina99,Komorebi,Luna
I Bananari,,simone volponi,Niko,simone volponi,massimopud
Armata Brancaleone,,Ella F.,AdStr,Ella F.,Skorpiòs
Living Stones,,Macleo,Ljuset,Macleo,Vincenzo Iennaco
Retroguardia Trash,,Andrea28,Eudes,Andrea28,AlexComan
Le TreMende,,Emy,Thea,Emy,sefora
Gli scrittori del quartierino,,Marty12,camparino,Marty12,Federico72
the 3 BrOthers,,AryaSophia,ettore70,AryaSophia,ITG


In [9]:
importlib.reload(melo)

# c'è una componente casuale, una volta pubblicati vanno fissati
#accoppiamenti=melo.calcola_accoppiamenti(1,formazioni,calendario,elo_squadre)
data=[['A', 1, 'D Euforiche', 'Kuno', 'queffe', 'Marcello', 'Kuno'],
 ['A', 1, 'La threesome', 'mina99', 'mina99', 'Komorebi', 'Luna'],
 ['B', 1, 'Green House', 'Bango Skank', 'Plata', 'Bango Skank', 'Joyopi'],
 ['B',
  1,
  'I Bananari',
  'simone volponi',
  'Niko',
  'massimopud',
  'simone volponi'],
 ['B',
  2,
  'Gli scrittori del quartierino',
  'Marty12',
  'camparino',
  'Marty12',
  'Federico72'],
 ['B', 2, 'the 3 BrOthers', 'AryaSophia', 'ettore70', 'ITG', 'AryaSophia'],
 ['C', 1, 'Living Stones', 'Macleo', 'Ljuset', 'Macleo', 'Vincenzo Iennaco'],
 ['C', 1, 'Retroguardia Trash', 'Andrea28', 'Eudes', 'Andrea28', 'AlexComan'],
 ['C', 2, 'Le TreMende', 'Emy', 'Thea', 'Emy', 'sefora'],
 ['C', 2, 'Armata Brancaleone', 'Ella F.', 'AdStr', 'Ella F.', 'Skorpiòs']]

accoppiamenti=pd.DataFrame(data,columns=["girone","partita","squadra","capitano",
                                         "riserva","match 1",
                                         "match 2"]).set_index(["girone","partita"])
accoppiamenti


squadra        capitano    riserva  \
girone partita                                                             
A      1                          D Euforiche            Kuno     queffe   
       1                         La threesome          mina99     mina99   
B      1                          Green House     Bango Skank      Plata   
       1                           I Bananari  simone volponi       Niko   
       2        Gli scrittori del quartierino         Marty12  camparino   
       2                       the 3 BrOthers      AryaSophia   ettore70   
C      1                        Living Stones          Macleo     Ljuset   
       1                   Retroguardia Trash        Andrea28      Eudes   
       2                          Le TreMende             Emy       Thea   
       2                   Armata Brancaleone         Ella F.      AdStr   

                    match 1           match 2  
girone partita                                 
A      1           Marcello              Kuno  
       1           Komorebi              Luna  
B      1        Bango Skank            Joyopi  
       1         massimopud    simone volponi  
       2            Marty12        Federico72  
       2                ITG        AryaSophia  
C      1             Macleo  Vincenzo Iennaco  
       1           Andrea28         AlexComan  
       2                Emy            sefora  
       2            Ella F.          Skorpiòs

In [10]:
# Codice html per il post di riepilogo sugli accoppiamenti

prev_girone=""
for girone,partita in accoppiamenti.index.unique():
    if prev_girone!=girone:
        print("<p>&nbsp;</p>")
        print("<p><strong>Girone %s</strong></p>" % girone)
        prev_girone=girone
    p1=accoppiamenti.loc[girone,partita].iloc[0]
    p2=accoppiamenti.loc[girone,partita].iloc[1]
    print("<p>%d. %s - %s</p>" % (partita,p1["squadra"],p2["squadra"]) )
    print("<p style='margin-left: 40px;'>match 1: %s - %s</p>" % (p1["match 1"],p2["match 1"]))
    print("<p style='margin-left: 40px;'>match 2: %s - %s</p>" % (p1["match 2"],p2["match 2"]))
    

<p>&nbsp;</p>
<p><strong>Girone A</strong></p>
<p>1. D Euforiche - La threesome</p>
<p style='margin-left: 40px;'>match 1: Marcello - Komorebi</p>
<p style='margin-left: 40px;'>match 2: Kuno - Luna</p>
<p>&nbsp;</p>
<p><strong>Girone B</strong></p>
<p>1. Green House - I Bananari</p>
<p style='margin-left: 40px;'>match 1: Bango Skank - massimopud</p>
<p style='margin-left: 40px;'>match 2: Joyopi - simone volponi</p>
<p>2. Gli scrittori del quartierino - the 3 BrOthers</p>
<p style='margin-left: 40px;'>match 1: Marty12 - ITG</p>
<p style='margin-left: 40px;'>match 2: Federico72 - AryaSophia</p>
<p>&nbsp;</p>
<p><strong>Girone C</strong></p>
<p>1. Living Stones - Retroguardia Trash</p>
<p style='margin-left: 40px;'>match 1: Macleo - Andrea28</p>
<p style='margin-left: 40px;'>match 2: Vincenzo Iennaco - AlexComan</p>
<p>2. Le TreMende - Armata Brancaleone</p>
<p style='margin-left: 40px;'>match 1: Emy - Ella F.</p>
<p style='margin-left: 40px;'>match 2: sefora - Skorpiòs</p>


## Fase 2. Creazione del tabellone per voti e controlli

In [11]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

# togliere quando c'è la versione definitiva delle formazioni
old_data=s.get_giornata(1)
#old_data=None
s.set_giornata(1,accoppiamenti,gironi,old_data)


### Tabella ammissioni

In [12]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
ammissioni=s.get_giornata(1)
ammissioni=ammissioni[["base","verifiche"]]

old_cols=ammissioni.columns.tolist()
new_cols=[c[1] for c in old_cols]
new_cols

ammissioni.columns=new_cols
a=accoppiamenti.reset_index().set_index("match 1")[["girone","partita","squadra","capitano","riserva"]]
a["match"]=1
a
b=accoppiamenti.reset_index().set_index("match 2")[["girone","partita","squadra","capitano","riserva"]]
b["match"]=2

ammissioni=ammissioni.merge(a.append(b),left_index=True,right_index=True)
new_index=["girone","partita","match","squadra"]
new_columns=["index","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]

ammissioni=ammissioni.reset_index().set_index(new_index).sort_index()[new_columns]
new_columns=["titolare","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]
ammissioni.columns=new_columns

In [13]:
def calc_ammissioni(row):
    row["sostituzione"]=""
    if row["ingresso riserva"]:
        sost="("+row["riserva"]+" da "+row["ingresso riserva"]+")"
        row["sostituzione"]=sost
    row["punti penalità"]=""
    if int(row["penalità"])==0: return row
    if abs(int(row["penalità"]))==1:
        row["punti penalità"]="1 punto di penalità"
    else:
        row["punti penalità"]="%d punti di penalità" % abs(int(row["penalità"]))
    return row

ammissioni=ammissioni.apply(calc_ammissioni,axis=1)
ammissioni

titolare  \
girone partita match squadra                                           
A      1       1     D Euforiche                            Marcello   
                     La threesome                           Komorebi   
               2     D Euforiche                                Kuno   
                     La threesome                               Luna   
B      1       1     Green House                         Bango Skank   
                     I Bananari                           massimopud   
               2     Green House                              Joyopi   
                     I Bananari                       simone volponi   
       2       1     Gli scrittori del quartierino           Marty12   
                     the 3 BrOthers                              ITG   
               2     Gli scrittori del quartierino        Federico72   
                     the 3 BrOthers                       AryaSophia   
C      1       1     Living Stones                            Macleo   
                     Retroguardia Trash                     Andrea28   
               2     Living Stones                  Vincenzo Iennaco   
                     Retroguardia Trash                    AlexComan   
       2       1     Armata Brancaleone                      Ella F.   
                     Le TreMende                                 Emy   
               2     Armata Brancaleone                     Skorpiòs   
                     Le TreMende                              sefora   

                                                      riserva  \
girone partita match squadra                                    
A      1       1     D Euforiche                       queffe   
                     La threesome                      mina99   
               2     D Euforiche                       queffe   
                     La threesome                      mina99   
B      1       1     Green House                        Plata   
                     I Bananari                          Niko   
               2     Green House                        Plata   
                     I Bananari                          Niko   
       2       1     Gli scrittori del quartierino  camparino   
                     the 3 BrOthers                  ettore70   
               2     Gli scrittori del quartierino  camparino   
                     the 3 BrOthers                  ettore70   
C      1       1     Living Stones                     Ljuset   
                     Retroguardia Trash                 Eudes   
               2     Living Stones                     Ljuset   
                     Retroguardia Trash                 Eudes   
       2       1     Armata Brancaleone                 AdStr   
                     Le TreMende                         Thea   
               2     Armata Brancaleone                 AdStr   
                     Le TreMende                         Thea   

                                                   ingresso riserva  \
girone partita match squadra                                          
A      1       1     D Euforiche                                      
                     La threesome                                     
               2     D Euforiche                                      
                     La threesome                                     
B      1       1     Green House                                      
                     I Bananari                                       
               2     Green House                                      
                     I Bananari                                       
       2       1     Gli scrittori del quartierino                    
                     the 3 BrOthers                                   
               2     Gli scrittori del quartierino                    
                     the 3 BrOthers                                   
C      1       1     Living Stones                

In [14]:

#old_cols=ammissioni.columns.tolist()
#new_cols=[c[1] for c in old_cols]
#new_cols

#ammissioni.columns=new_cols

mu.to_table_html_multi(ammissioni[["titolare","titolo",
                                   "lunghezza"]])

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>match</th>
<th>squadra</th>
<th>titolare</th>
<th>titolo</th>
<th>lunghezza</th>
</tr>
<tr>
<th rowspan='4'>A</th>
<th rowspan='8'>1</th>
<th rowspan='2'>1</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>Marcello</td>
<td style='text-align:center;'>In attesa</td>
<td style='text-align:center;'>5820</td>
</tr>
<tr>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>Komorebi</td>
<td style='text-align:center;'>Assalto alla fortezza</td>
<td style='text-align:center;'>7149</td>
</tr>
<tr>
<th rowspan='2'>2</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>Kuno</td>
<td style='text-align:center;'>Vergogna. Scuse. Catapulta.</td>
<td style='text-align:center;'>7988</td>
</tr>
<tr>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>Luna</td>
<td style='text-align:center;'>I macellai di erbafiore</td>
<td style='text-align:center;'>7947</

## Fase 3. Recupero dati

In [40]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
voti=s.get_giornata(1)


In [41]:
voti

base  \
                 ingresso riserva   
titolare                            
Marcello                            
Kuno                                
Komorebi                            
Luna                                
Bango Skank                         
Joyopi                              
massimopud                          
simone volponi                      
Marty12                             
Federico72                          
ITG                                 
AryaSophia                          
Macleo                              
Vincenzo Iennaco                    
Andrea28                            
AlexComan                           
Emy                                 
sefora                              
Ella F.                             
Skorpiòs                            

                                                               verifiche      \
                                                        titolo lunghezza tag   
titolare                                                                       
Marcello                                             In attesa      5820   x   
Kuno                               Vergogna. Scuse. Catapulta.      7988   x   
Komorebi                                 Assalto alla fortezza      7149   x   
Luna                                   I macellai di erbafiore      7947   x   
Bango Skank               La Catapulta dei Cavalieri del Leone      7922   x   
Joyopi            Dino Saurini e le care buon vecchie amicizie      7903   x   
massimopud                                 La grande catapulta      6084   x   
simone volponi                                     Un bel tiro      5914   x   
Marty12                               La catapulta delle anime      7464   x   
Federico72          Kaiten - Catapultati in braccio alla morte      5344   x   
ITG                                  Lui beve per... ricordare      5416   x   
AryaSophia                                 Areth, la prescelta      5061   x   
Macleo                                         Manna dal cielo      7944   x   
Vincenzo Iennaco                                Sotto assedio       7608   x   
Andrea28                                 Catapulta leggendaria      7999   x   
AlexComan                             Il sogno di ogni gallina      7216   x   
Emy                               La notte delle teste cadenti      7909   x   
sefora                                          È quasi Natale      7861   x   
Ella F.                                              Katapulta      6934   x   
Skorpiòs                                            Il maniero      6888   x   

                                                                       A  \
                 link link commento commento valido penalità D Euforiche   
titolare                                                                   
Marcello            x             x               x        0           0   
Kuno                x             x               x        0           0   
Komorebi            x             x               x        0           0   
Luna                x             x               x        0           0   
Bango Skank         x             x               x        0           0   
Joyopi              x             x               x        0           1   
massimopud          x             x               x        0           2   
simone volponi      x             x               x        0           1   
Marty12             x             x               x        0           0   
Federico72          x             x               x        0           2   
ITG                 x             x               x        0           2   
AryaSophia          x             x               x        0           0   
Macleo              x             x               x        0           2   
Vincenzo Iennaco    x             x               x        0           1   
Andrea28            x             x         

In [34]:
### Simulazione di riempimento del tabellone - da togliere quando ci sono i voti definitivi
#importlib.reload(mu)

#voti["base","titolo"]=voti["base","titolo"].apply(mu.random_title)
#voti["base","ingresso riserva"]=voti["base","ingresso riserva"].apply(mu.random_riserva)
#voti["verifiche","lunghezza"]=voti["verifiche","lunghezza"].apply(mu.random_len)
#voti["verifiche","link"]=voti["verifiche","link"].apply(mu.random_verifica)
#voti["verifiche","link commento"]=voti["verifiche","link commento"].apply(mu.random_verifica)
#voti["verifiche","tag"]=voti["verifiche","tag"].apply(mu.random_verifica)
#voti["verifiche","commento valido"]=voti["verifiche","commento valido"].apply(mu.random_verifica)

#for G,altri in [ ("A",[("B",1),("B",2),("C",1),("C",2)]),
#                 ("B",[("A",1),("C",1),("C",2)]),
#                 ("C",[("A",1),("B",1),("B",2)]) ]:
#    for sq in gironi[gironi[G]!="(riposo)"][G]:
#        for gir,part in altri:
#            for m in ["match 1","match 2"]:
#                giocatore_1=accoppiamenti.loc[gir,part][m].iloc[0]
#                giocatore_2=accoppiamenti.loc[gir,part][m].iloc[1]
#                goal_1=random.randint(0,2)
#                goal_2=2-goal_1
#                voti.set_value(giocatore_1,(G,sq),goal_1)
#                voti.set_value(giocatore_2,(G,sq),goal_2)

#voti["verifiche","penalità"]=voti["verifiche","lunghezza"].apply(mu.calc_penalita_lungh)
#for k in ["tag","link","link commento","commento valido"]:
#    voti["verifiche","penalità"]+=voti["verifiche",k].apply(mu.calc_penalita)


In [42]:
voti["verifiche","penalità"]=pd.to_numeric(voti["verifiche","penalità"],downcast="signed")
voti["base","goal giocatore"]=voti[["A","B","C"]].sum(axis=1,level=1).sum(axis=1)+voti["verifiche","penalità"]
voti["base","goal giocatore"]=pd.to_numeric(voti["base","goal giocatore"],downcast="signed")
voti

base  \
                 ingresso riserva   
titolare                            
Marcello                            
Kuno                                
Komorebi                            
Luna                                
Bango Skank                         
Joyopi                              
massimopud                          
simone volponi                      
Marty12                             
Federico72                          
ITG                                 
AryaSophia                          
Macleo                              
Vincenzo Iennaco                    
Andrea28                            
AlexComan                           
Emy                                 
sefora                              
Ella F.                             
Skorpiòs                            

                                                               verifiche      \
                                                        titolo lunghezza tag   
titolare                                                                       
Marcello                                             In attesa      5820   x   
Kuno                               Vergogna. Scuse. Catapulta.      7988   x   
Komorebi                                 Assalto alla fortezza      7149   x   
Luna                                   I macellai di erbafiore      7947   x   
Bango Skank               La Catapulta dei Cavalieri del Leone      7922   x   
Joyopi            Dino Saurini e le care buon vecchie amicizie      7903   x   
massimopud                                 La grande catapulta      6084   x   
simone volponi                                     Un bel tiro      5914   x   
Marty12                               La catapulta delle anime      7464   x   
Federico72          Kaiten - Catapultati in braccio alla morte      5344   x   
ITG                                  Lui beve per... ricordare      5416   x   
AryaSophia                                 Areth, la prescelta      5061   x   
Macleo                                         Manna dal cielo      7944   x   
Vincenzo Iennaco                                Sotto assedio       7608   x   
Andrea28                                 Catapulta leggendaria      7999   x   
AlexComan                             Il sogno di ogni gallina      7216   x   
Emy                               La notte delle teste cadenti      7909   x   
sefora                                          È quasi Natale      7861   x   
Ella F.                                              Katapulta      6934   x   
Skorpiòs                                            Il maniero      6888   x   

                                                                       A  \
                 link link commento commento valido penalità D Euforiche   
titolare                                                                   
Marcello            x             x               x        0           0   
Kuno                x             x               x        0           0   
Komorebi            x             x               x        0           0   
Luna                x             x               x        0           0   
Bango Skank         x             x               x        0           0   
Joyopi              x             x               x        0           1   
massimopud          x             x               x        0           2   
simone volponi      x             x               x        0           1   
Marty12             x             x               x        0           0   
Federico72          x             x               x        0           2   
ITG                 x             x               x        0           2   
AryaSophia          x             x               x        0           0   
Macleo              x             x               x        0           2   
Vincenzo Iennaco    x             x               x        0           1   
Andrea28            x             x         

## Calcolo dei risultati

### Risultati della prima giornata

In [43]:
risultati=accoppiamenti.copy()

risultati=risultati.reset_index().set_index("squadra")
risultati["match 1 goal"]=0
risultati["match 1 riserva"]=False
risultati["match 2 goal"]=0
risultati["match 2 riserva"]=False
risultati["goal"]=0
risultati["penalità"]=0


for sq in risultati.index:
    m1=risultati.loc[sq]["match 1"]
    m2=risultati.loc[sq]["match 2"]
    goal_1=voti.loc[m1]["base","goal giocatore"]
    goal_2=voti.loc[m2]["base","goal giocatore"]
    pen_1=voti.loc[m1]["verifiche","penalità"]
    pen_2=voti.loc[m2]["verifiche","penalità"]
    risultati.set_value(sq,"match 1 goal",goal_1)
    risultati.set_value(sq,"match 2 goal",goal_2)
    risultati.set_value(sq,"goal",goal_1+goal_2)
    risultati.set_value(sq,"penalità",pen_1+pen_2)
    ris_1=bool(voti.loc[m1]["base","ingresso riserva"])
    risultati.set_value(sq,"match 1 riserva",ris_1)
    if ris_1: continue
    risultati.set_value(sq,"match 2 riserva",bool(voti.loc[m2]["base","ingresso riserva"]))
    
risultati=risultati.reset_index().set_index(["girone","partita"])
sc_dati=[]
for girone,partita in risultati.index.unique():
    squadra_1=risultati.loc[girone,partita]["squadra"].iloc[0]
    squadra_2=risultati.loc[girone,partita]["squadra"].iloc[1]
    goal_1=risultati.loc[girone,partita]["goal"].iloc[0]
    goal_2=risultati.loc[girone,partita]["goal"].iloc[1]
    pen_1=risultati.loc[girone,partita]["penalità"].iloc[0]
    pen_2=risultati.loc[girone,partita]["penalità"].iloc[1]
    m1_goal_1=risultati.loc[girone,partita]["match 1 goal"].iloc[0]
    m1_goal_2=risultati.loc[girone,partita]["match 1 goal"].iloc[1]
    m2_goal_1=risultati.loc[girone,partita]["match 2 goal"].iloc[0]
    m2_goal_2=risultati.loc[girone,partita]["match 2 goal"].iloc[1]
    sc_dati.append([girone,partita,squadra_1,squadra_2,goal_1,goal_2,
                    m1_goal_1,m1_goal_2,m2_goal_1,m2_goal_2,pen_1,pen_2])

risultati=risultati.reset_index().set_index(["girone","partita","squadra"])

schedina=pd.DataFrame(sc_dati,columns=["girone","partita","squadra 1","squadra 2",
                                       "goal squadra 1","goal squadra 2",
                                       "match 1 squadra 1","match 1 squadra 2",
                                       "match 2 squadra 1","match 2 squadra 2",
                                       "penalità squadra 1","penalità squadra 2"])

out_schedina=schedina.copy()
out_schedina["titolo"]=out_schedina["squadra 1"]+" - "+out_schedina["squadra 2"]
out_schedina["risultato"]=( out_schedina["goal squadra 1"].apply(str)
                            +" - "+out_schedina["goal squadra 2"].apply(str) )
out_schedina["match 1"]=( out_schedina["match 1 squadra 1"].apply(str)
                          +" - "+out_schedina["match 1 squadra 2"].apply(str) )
out_schedina["match 2"]=( out_schedina["match 2 squadra 1"].apply(str)
                          +" - "+out_schedina["match 2 squadra 2"].apply(str) )
out_schedina=out_schedina.set_index(["girone","partita"])

out_schedina=out_schedina[["titolo","match 1","match 2","risultato"]]


risultati

capitano    riserva  \
girone partita squadra                                                    
A      1       D Euforiche                              Kuno     queffe   
               La threesome                           mina99     mina99   
B      1       Green House                       Bango Skank      Plata   
               I Bananari                     simone volponi       Niko   
       2       Gli scrittori del quartierino         Marty12  camparino   
               the 3 BrOthers                     AryaSophia   ettore70   
C      1       Living Stones                          Macleo     Ljuset   
               Retroguardia Trash                   Andrea28      Eudes   
       2       Le TreMende                               Emy       Thea   
               Armata Brancaleone                    Ella F.      AdStr   

                                                  match 1           match 2  \
girone partita squadra                                                        
A      1       D Euforiche                       Marcello              Kuno   
               La threesome                      Komorebi              Luna   
B      1       Green House                    Bango Skank            Joyopi   
               I Bananari                      massimopud    simone volponi   
       2       Gli scrittori del quartierino      Marty12        Federico72   
               the 3 BrOthers                         ITG        AryaSophia   
C      1       Living Stones                       Macleo  Vincenzo Iennaco   
               Retroguardia Trash                Andrea28         AlexComan   
       2       Le TreMende                            Emy            sefora   
               Armata Brancaleone                 Ella F.          Skorpiòs   

                                              match 1 goal  match 1 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                               8            False   
               La threesome                              8            False   
B      1       Green House                               7            False   
               I Bananari                                7            False   
       2       Gli scrittori del quartierino             2            False   
               the 3 BrOthers                           12            False   
C      1       Living Stones                             3            False   
               Retroguardia Trash                       11            False   
       2       Le TreMende                               0            False   
               Armata Brancaleone                       14            False   

                                              match 2 goal  match 2 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                              13            False   
               La threesome                              3            False   
B      1       Green House                               5            False   
               I Bananari                                9            False   
       2       Gli scrittori del quartierino            14            False   
               the 3 BrOthers                            0            False   
C      1       Living Stones                             4            False   
               Retroguardia Trash                       10            False   
       2       Le TreMende                              10            False   
               Armata Brancaleone                        4            False   

                                              goal  penalità  
girone partita squadra                                        
A      1       D Euforiche                      21         0  
               La threesome                     11         0  
B      1       Green House                      12  

In [37]:
schedina

,girone,partita,squadra 1,squadra 2,goal squadra 1,goal squadra 2,match 1 squadra 1,match 1 squadra 2,match 2 squadra 1,match 2 squadra 2,penalità squadra 1,penalità squadra 2
0,A,1,D Euforiche,La threesome,21,11,8,8,13,3,0,0
1,B,1,Green House,I Bananari,12,16,7,7,5,9,0,0
2,B,2,Gli scrittori del quartierino,the 3 BrOthers,16,12,2,12,14,0,0,0
3,C,1,Living Stones,Retroguardia Trash,7,21,3,11,4,10,0,0
4,C,2,Le TreMende,Armata Brancaleone,10,18,0,14,10,4,0,0


In [38]:
out_schedina

titolo match 1  \
girone partita                                                           
A      1                            D Euforiche - La threesome   8 - 8   
B      1                              Green House - I Bananari   7 - 7   
       2        Gli scrittori del quartierino - the 3 BrOthers  2 - 12   
C      1                    Living Stones - Retroguardia Trash  3 - 11   
       2                      Le TreMende - Armata Brancaleone  0 - 14   

               match 2 risultato  
girone partita                    
A      1        13 - 3   21 - 11  
B      1         5 - 9   12 - 16  
       2        14 - 0   16 - 12  
C      1        4 - 10    7 - 21  
       2        10 - 4   10 - 18

In [39]:
# output per il post dei risultati

mu.to_table_html_multi(out_schedina)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>titolo</th>
<th>match 1</th>
<th>match 2</th>
<th>risultato</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche - La threesome</td>
<td style='text-align:center;'>8 - 8</td>
<td style='text-align:center;'>13 - 3</td>
<td style='text-align:center;'>21 - 11</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House - I Bananari</td>
<td style='text-align:center;'>7 - 7</td>
<td style='text-align:center;'>5 - 9</td>
<td style='text-align:center;'>12 - 16</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Gli scrittori del quartierino - the 3 BrOthers</td>
<td style='text-align:center;'>2 - 12</td>
<td style='text-align:center;'>14 - 0</td>
<td style='text-align:center;'>16 - 12</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones - Retroguardia Trash</td>

### Classifica


Ordine dei criteri:
* classifiche avulse (scontri diretti);
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti nell'ultimo incontro;
* numero di reti segnate nell'ultimo incontro;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.

Nella prima giornata non ci sono classifiche avulse e l'unico incontro è anche l'ultimo, quindi si considerano solo:
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.


In [23]:
q=schedina.copy()

#q.merge(q,left_on="squadra 1",right_on="squadra 2",suffixes=(" sq1"," sq2"),how="outer")

a=q[["girone","squadra 1","goal squadra 1","goal squadra 2",
     "penalità squadra 1","penalità squadra 2"]].copy()
b=q[["girone","squadra 2","goal squadra 2","goal squadra 1",
     "penalità squadra 2","penalità squadra 1"]].copy()

a.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]
b.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]

classifica=a.append(b).set_index(["girone","squadra"]).sort_index()

classifica["goal netti fatti"]=classifica["goal fatti"]-classifica["penalità fatte"]
classifica["goal netti subiti"]=classifica["goal subiti"]-classifica["penalità subite"]

classifica["differenza reti"]=classifica["goal fatti"]-classifica["goal subiti"]
classifica["differenza reti netta"]=classifica["goal netti fatti"]-classifica["goal netti subiti"]

def calc_vittorie(row):
    row["vittorie"]=0
    row["sconfitte"]=0
    row["pareggi"]=0
    if row["goal fatti"]>row["goal subiti"]:
        row["vittorie"]=1
    elif row["goal fatti"]<row["goal subiti"]:
        row["sconfitte"]=1
    else:
        row["pareggi"]=1
    row["punti"]=3*row["vittorie"]+row["pareggi"]
    return row

classifica=classifica.apply(calc_vittorie,axis=1)


df=pd.DataFrame([["A","Gli ultimi",0]],columns=["girone","squadra","goal fatti"])
df=df.set_index(["girone","squadra"])
classifica=classifica.append(df).fillna(0).sort_index()
classifica=classifica[["punti","vittorie","pareggi","sconfitte","differenza reti",
                   "goal fatti","goal subiti",
                   "differenza reti netta",
                   "goal netti fatti","goal netti subiti","penalità fatte","penalità subite"]]
classifica=classifica.astype(int)
classifica=classifica.sort_values(by=["punti","differenza reti","goal fatti","vittorie",
                                  "differenza reti netta","goal netti fatti"],
                              ascending=False).sort_index(level="girone",
                                                          kind="mergesort",
                                                          sort_remaining=False)
classifica

punti  vittorie  pareggi  sconfitte  \
girone squadra                                                              
A      D Euforiche                        3         1        0          0   
       Gli ultimi                         0         0        0          0   
       La threesome                       0         0        0          1   
B      Gli scrittori del quartierino      3         1        0          0   
       I Bananari                         3         1        0          0   
       Green House                        0         0        0          1   
       the 3 BrOthers                     0         0        0          1   
C      Retroguardia Trash                 3         1        0          0   
       Armata Brancaleone                 3         1        0          0   
       Le TreMende                        0         0        0          1   
       Living Stones                      0         0        0          1   

                                      differenza reti  goal fatti  \
girone squadra                                                      
A      D Euforiche                                 10          21   
       Gli ultimi                                   0           0   
       La threesome                               -10          11   
B      Gli scrittori del quartierino                4          16   
       I Bananari                                   4          16   
       Green House                                 -4          12   
       the 3 BrOthers                              -4          12   
C      Retroguardia Trash                          14          21   
       Armata Brancaleone                           8          18   
       Le TreMende                                 -8          10   
       Living Stones                              -14           7   

                                      goal subiti  differenza reti netta  \
girone squadra                                                             
A      D Euforiche                             11                     10   
       Gli ultimi                               0                      0   
       La threesome                            21                    -10   
B      Gli scrittori del quartierino           12                      4   
       I Bananari                              12                      4   
       Green House                             16                     -4   
       the 3 BrOthers                          16                     -4   
C      Retroguardia Trash                       7                     14   
       Armata Brancaleone                      10                      8   
       Le TreMende                             18                     -8   
       Living Stones                           21                    -14   

                                      goal netti fatti  goal netti subiti  \
girone squadra                                                              
A      D Euforiche                                  21                 11   
       Gli ultimi                                    0                  0   
       La threesome                                 11                 21   
B      Gli scrittori del quartierino                16                 12   
       I Bananari                                   16                 12   
       Green House                                  12                 16   
       the 3 BrOthers                               12                 16   
C      Retroguardia Trash                           21                  7   
       Armata Brancaleone                           18                 10   
       Le TreMende                                  10                 18   
       Living Stones                                 7                 21   

                                      penalità fatte  penalità subite  
girone squadra                                                         
A  

In [24]:
## Output per il post della classifica

mu.to_table_html_multi(classifica)

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>squadra</th>
<th>punti</th>
<th>vittorie</th>
<th>pareggi</th>
<th>sconfitte</th>
<th>differenza reti</th>
<th>goal fatti</th>
<th>goal subiti</th>
<th>differenza reti netta</th>
<th>goal netti fatti</th>
<th>goal netti subiti</th>
<th>penalità fatte</th>
<th>penalità subite</th>
</tr>
<tr>
<th rowspan='3'>A</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>3</td>
<td style='text-align:center;'>1</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>10</td>
<td style='text-align:center;'>21</td>
<td style='text-align:center;'>11</td>
<td style='text-align:center;'>10</td>
<td style='text-align:center;'>21</td>
<td style='text-align:center;'>11</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
</tr>
<tr>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>

### Nuovi punteggi Elo

In [25]:
importlib.reload(melo)

q=risultati.copy()

def calc_giocatore(row,match):
    riserva=row["riserva"]
    giocatore=row["match %d" % match]
    sostituzione=row["match %d riserva" % match]
    if sostituzione: return riserva
    return giocatore

def calc_giocatore_1(row): return calc_giocatore(row,1)
def calc_giocatore_2(row): return calc_giocatore(row,2)

q["giocatore 1"]=q.apply(calc_giocatore_1,axis=1)
q["giocatore 2"]=q.apply(calc_giocatore_2,axis=1)

a=q[["giocatore 1","match 1 goal"]].copy()
a.columns=["giocatore","goal"]
a["match"]="1"

b=q[["giocatore 2","match 2 goal"]].copy()
b.columns=["giocatore","goal"]
b["match"]="2"

q=a.append(b).reset_index().set_index(["girone","partita","match"])
q=q.merge(q,how="outer",left_index=True,right_index=True,suffixes=[" 1"," 2"])
q=q[q["giocatore 1"]!=q["giocatore 2"]]

elo=q[["squadra 1","giocatore 1","giocatore 2","goal 1","goal 2"]]
elo.columns=["squadra","giocatore","avversario","goal fatti","goal subiti"]
elo=elo.reset_index().set_index(["girone","squadra","giocatore"]).sort_index()
elo=elo[["avversario","goal fatti","goal subiti"]].copy()
elo["elo precedente"]=1500
elo["elo avversario"]=1500

def calcola_punteggi(row):
    exp_a,exp_b=melo.punteggi_attesi(row["elo precedente"],row["elo avversario"])
    score_a,score_b=melo.punteggi_reali(row["goal fatti"],row["goal subiti"])
    row["punteggio atteso"]=exp_a
    row["punteggio reale"]=score_a
    return row

elo=elo.apply(calcola_punteggi,axis=1)

elo["scostamento"]=elo["punteggio reale"]-elo["punteggio atteso"]

elo=elo.reset_index().set_index(["giocatore"])
elo

,girone,squadra,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento
giocatore,,,,,,,,,,
Kuno,A,D Euforiche,Luna,13,3,1500,1500,0.5,0.812500,0.312500
Marcello,A,D Euforiche,Komorebi,8,8,1500,1500,0.5,0.500000,0.000000
Komorebi,A,La threesome,Marcello,8,8,1500,1500,0.5,0.500000,0.000000
Luna,A,La threesome,Kuno,3,13,1500,1500,0.5,0.187500,-0.312500
Federico72,B,Gli scrittori del quartierino,AryaSophia,14,0,1500,1500,0.5,1.000000,0.500000
Marty12,B,Gli scrittori del quartierino,ITG,2,12,1500,1500,0.5,0.142857,-0.357143
Bango Skank,B,Green House,massimopud,7,7,1500,1500,0.5,0.500000,0.000000
Joyopi,B,Green House,simone volponi,5,9,1500,1500,0.5,0.357143,-0.142857
massimopud,B,I Bananari,Bango Skank,7,7,1500,1500,0.5,0.500000,0.000000


In [26]:
elo_s=classifica.copy().reset_index().set_index("squadra")[["goal fatti","goal subiti"]]
elo_s["elo precedente"]=1500 # media dei giocatori
elo_s["elo avversario"]=1500 # media dei giocatori
elo_s=elo_s.apply(calcola_punteggi,axis=1)
elo_s["scostamento squadra"]=elo_s["punteggio reale"]-elo_s["punteggio atteso"]
elo_s=elo_s.reset_index().merge(elo[["girone","squadra"]], on="squadra",how="left")
elo_s=elo_s.drop_duplicates()
elo_s["girone"]=elo_s["girone"].fillna("A")
elo_s

,squadra,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento squadra,girone
0,D Euforiche,21.0,11.0,1500.0,1500.0,0.5,0.656250,0.156250,A
2,Gli ultimi,0.0,0.0,1500.0,1500.0,0.5,0.500000,0.000000,A
3,La threesome,11.0,21.0,1500.0,1500.0,0.5,0.343750,-0.156250,A
5,Gli scrittori del quartierino,16.0,12.0,1500.0,1500.0,0.5,0.571429,0.071429,B
7,I Bananari,16.0,12.0,1500.0,1500.0,0.5,0.571429,0.071429,B
9,Green House,12.0,16.0,1500.0,1500.0,0.5,0.428571,-0.071429,B
11,the 3 BrOthers,12.0,16.0,1500.0,1500.0,0.5,0.428571,-0.071429,B
13,Retroguardia Trash,21.0,7.0,1500.0,1500.0,0.5,0.750000,0.250000,C
15,Armata Brancaleone,18.0,10.0,1500.0,1500.0,0.5,0.642857,0.142857,C
17,Le TreMende,10.0,18.0,1500.0,1500.0,0.5,0.357143,-0.142857,C


In [27]:
q=elo_s.reset_index()[["girone","squadra","scostamento squadra"]]
#elo.merge(d,on="squadra")
q=squadre.reset_index().merge(q,on="squadra").set_index(["giocatore","squadra","girone"])
elo=elo.reset_index().set_index(["giocatore","squadra","girone"]).merge(q[["scostamento squadra"]],left_index=True,right_index=True,how="outer")
elo["goal fatti"]=pd.to_numeric(elo["goal fatti"].fillna(0),downcast="signed")
elo["goal subiti"]=pd.to_numeric(elo["goal subiti"].fillna(0),downcast="signed")
elo["elo precedente"]=pd.to_numeric(elo["elo precedente"].fillna(1500),downcast="signed")
elo["elo avversario"]=pd.to_numeric(elo["elo avversario"].fillna(1500),downcast="signed")
elo["punteggio atteso"]=elo["punteggio atteso"].fillna(0.5)
elo["punteggio reale"]=elo["punteggio reale"].fillna(0.5)
elo["scostamento"]=elo["scostamento"].fillna(0.0)
elo["avversario"]=elo["avversario"].fillna("(non giocato)")

elo=elo.reset_index().set_index(["giocatore"])

elo


,squadra,girone,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento,scostamento squadra
giocatore,,,,,,,,,,,
AdStr,Armata Brancaleone,C,(non giocato),0,0,1500,1500,0.5,0.500000,0.000000,0.142857
AlexComan,Retroguardia Trash,C,Vincenzo Iennaco,10,4,1500,1500,0.5,0.714286,0.214286,0.250000
Andrea28,Retroguardia Trash,C,Macleo,11,3,1500,1500,0.5,0.785714,0.285714,0.250000
AryaSophia,the 3 BrOthers,B,Federico72,0,14,1500,1500,0.5,0.000000,-0.500000,-0.071429
Bango Skank,Green House,B,massimopud,7,7,1500,1500,0.5,0.500000,0.000000,-0.071429
Ella F.,Armata Brancaleone,C,Emy,14,0,1500,1500,0.5,1.000000,0.500000,0.142857
Emy,Le TreMende,C,Ella F.,0,14,1500,1500,0.5,0.000000,-0.500000,-0.142857
Eudes,Retroguardia Trash,C,(non giocato),0,0,1500,1500,0.5,0.500000,0.000000,0.250000
Federico72,Gli scrittori del quartierino,B,AryaSophia,14,0,1500,1500,0.5,1.000000,0.500000,0.071429


In [28]:
def calc_K(dfcol,max_variazione):
    mean=dfcol.mean()
    sigma=dfcol.std()
    for alpha in np.linspace(0,6,601):
        if mlab.normpdf(alpha*sigma,mean,sigma)<=0.0001:
            break
    max_scostamento=alpha*sigma
    K=max_variazione/max_scostamento
    return K

def norm_elo(x):
    e=round(max(1450,x))
    return int(e)

KG=calc_K(elo["scostamento"],70)
KS=calc_K(elo["scostamento squadra"],20)


elo["elo nuovo"]=elo["elo precedente"]+KG*elo["scostamento"]+KS*elo["scostamento squadra"]
elo["elo nuovo"]=elo["elo nuovo"].apply(norm_elo)

elo[["elo nuovo"]].sort_values("elo nuovo",ascending=False)


,elo nuovo
giocatore,
Ella F.,1537
Federico72,1535
Andrea28,1526
Kuno,1525
AlexComan,1522
ITG,1522
simone volponi,1512
sefora,1510
Eudes,1507


In [29]:
a=gironi[gironi["A"]!="(riposo)"][["A"]].copy()
a.columns=["squadra"]
a["girone"]="A"
a

b=gironi[gironi["B"]!="(riposo)"][["B"]].copy()
b.columns=["squadra"]
b["girone"]="B"
b

c=gironi[gironi["C"]!="(riposo)"][["C"]].copy()
c.columns=["squadra"]
c["girone"]="C"
c

q=a.append(b).append(c).set_index("squadra")
q=q.merge(squadre,left_index=True,right_index=True,how="outer")
q=q.reset_index().set_index("giocatore")
q

elo_g=elo.merge(q,how="outer",
                left_index=True,right_index=True)
elo_g=elo_g[["squadra_y","elo nuovo"]].fillna(1500).sort_values("elo nuovo",
                                                                           ascending=False)
elo_g["elo nuovo"]=pd.to_numeric(elo_g["elo nuovo"],downcast="signed")
elo_g.columns=["squadra","elo"]
elo_g

,squadra,elo
giocatore,,
Ella F.,Armata Brancaleone,1537
Federico72,Gli scrittori del quartierino,1535
Andrea28,Retroguardia Trash,1526
Kuno,D Euforiche,1525
AlexComan,Retroguardia Trash,1522
ITG,the 3 BrOthers,1522
simone volponi,I Bananari,1512
sefora,Le TreMende,1510
Eudes,Retroguardia Trash,1507


In [30]:
mu.to_table_html(elo_g,with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra</th>
<th>elo</th>
</tr>
<tr>
<th>Ella F.</th>
<td style='text-align:center;'>Armata Brancaleone</td>
<td style='text-align:center;'>1537</td>
</tr>
<tr>
<th>Federico72</th>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
<td style='text-align:center;'>1535</td>
</tr>
<tr>
<th>Andrea28</th>
<td style='text-align:center;'>Retroguardia Trash</td>
<td style='text-align:center;'>1526</td>
</tr>
<tr>
<th>Kuno</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1525</td>
</tr>
<tr>
<th>AlexComan</th>
<td style='text-align:center;'>Retroguardia Trash</td>
<td style='text-align:center;'>1522</td>
</tr>
<tr>
<th>ITG</th>
<td style='text-align:center;'>the 3 BrOthers</td>
<td style='text-align:center;'>1522</td>
</tr>
<tr>
<th>simone volponi</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>1512</td>
</tr>
<tr>
<th>sefora</th>
<td s

## Salvataggio dei risultati

In [31]:
## CSV
label="giornata_1"

# formazioni
formazioni_csv="data/formazioni_"+label+".csv"
formazioni.to_csv(formazioni_csv,index_label="squadra")

# accoppiamenti
accoppiamenti_csv="data/accoppiamenti_"+label+".csv"
accoppiamenti.to_csv(accoppiamenti_csv)

# voti
voti_csv="data/voti_"+label+".csv"
voti.to_csv(voti_csv)

# risultati
risultati_csv="data/risultati_"+label+".csv"
risultati.to_csv(risultati_csv)

# schedina
schedina_csv="data/schedina_"+label+".csv"
schedina.to_csv(schedina_csv,index=False)

# classifica
classifica_csv="data/classifica_"+label+".csv"
classifica.to_csv(classifica_csv)

# elo
elo_csv="data/elo_"+label+".csv"
elo.to_csv(elo_csv)

# elo_g
elo_g_csv="data/elo_g_"+label+".csv"
elo_g.to_csv(elo_g_csv)

# ammissioni
ammissioni_csv="data/ammissioni_"+label+".csv"
ammissioni.to_csv(ammissioni_csv)

# elo_g
elo_s_csv="data/elo_s_"+label+".csv"
elo_s.to_csv(elo_s_csv)